# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [29]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [30]:
orders = pd.read_csv("Orders.csv")
orders.head()


,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [31]:
#GROUPBY SEM COSTOMERID COMO INDEX
ordersgroup = orders.groupby('CustomerID', as_index=False).agg({'amount_spent':'sum'}).sort_values(by='amount_spent')
ordersgroup

,CustomerID,amount_spent
685,13256,0.00
3218,16738,3.75
1794,14792,6.20
3015,16454,6.90
4099,17956,12.75
...,...,...
1880,14911,143825.06
3009,16446,168472.50
3729,17450,194550.79
4202,18102,259657.30


In [32]:
#Sub Problem 1: How to aggregate the amount_spent for unique customers?

ordersgroup = orders.groupby('CustomerID')['amount_spent'].sum().sort_values()

ordersgroup = pd.DataFrame(ordersgroup)

ordersgroup






,amount_spent
CustomerID,
13256,0.00
16738,3.75
14792,6.20
16454,6.90
17956,12.75
...,...
14911,143825.06
16446,168472.50
17450,194550.79


In [33]:
ordersgroup.describe()

,amount_spent
count,4339.000000
mean,2053.793018
std,8988.248381
min,0.000000
25%,307.245000
50%,674.450000
75%,1661.640000
max,280206.020000


In [34]:
#Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?
#FIRST SOLUTION
ordersquantil = orders.groupby('CustomerID')['amount_spent'].sum()
ordersquantil0 = ordersquantil.describe().transpose()
ordersquantil0

count      4339.000000
mean       2053.793018
std        8988.248381
min           0.000000
25%         307.245000
50%         674.450000
75%        1661.640000
max      280206.020000
Name: amount_spent, dtype: float64

In [35]:
#Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?
#SECOND SOLUTION

ordersquantile = orders.groupby('CustomerID')['amount_spent'].sum().quantile([0, 0.25, 0.5, 0.75, 0.95, 1])
ordersquantile

0.00         0.000
0.25       307.245
0.50       674.450
0.75      1661.640
0.95      5840.182
1.00    280206.020
Name: amount_spent, dtype: float64

In [36]:
#Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?
#THIRD SOLUTION


# 75th Percentile
def q75(x):
            return x.quantile(0.75)

# 95th Percentile
def q95(x):
            return x.quantile(0.95)


ordersquantile2 = orders.groupby(['CustomerID'])['amount_spent'].sum().agg([q75, q95, 'max'])
ordersquantile2

q75      1661.640
q95      5840.182
max    280206.020
Name: amount_spent, dtype: float64

In [37]:
#q95 - 5840.182
VIP_Customers = ordersgroup[ordersgroup['amount_spent'] > 5840.182]

VIP_Customers


,amount_spent
CustomerID,
13534,5870.08
13209,5891.61
14944,5900.25
14415,5900.28
14505,5905.64
...,...
14911,143825.06
16446,168472.50
17450,194550.79


In [38]:
#q75 - 1661.640
#q95 - 5840.182

condition = (ordersgroup['amount_spent'] > 1661.640) & (ordersgroup['amount_spent'] < 5840.182)
Preferred_Customers = ordersgroup[condition]
Preferred_Customers

,amount_spent
CustomerID,
13012,1661.84
12530,1662.28
12912,1662.30
16115,1667.97
17656,1674.69
...,...
13178,5725.47
17686,5739.46
15218,5756.89


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [39]:
countrygroup = orders.groupby(['Country','CustomerID'])['amount_spent'].sum()
countrygroup = pd.DataFrame(countrygroup)


countryVIP= countrygroup[countrygroup['amount_spent'] > 5840.182]

pd.set_option('display.max_rows', 216)

countryVIP




amount_spent
Country         CustomerID              
Australia       12415          124914.53
Channel Islands 14936            8137.02
Cyprus          12359            6372.58
Denmark         12435            7829.89
EIRE            14156          117379.63
                14911          143825.06
Finland         12428            7956.46
France          12536           12601.83
                12567            9401.28
                12583            7281.38
                12637            6021.90
                12678           17628.46
                12681           13787.54
                12682           12288.22
                12683            8476.57
                12731           18895.91
Germany         12471           19824.05
                12472            6572.11
                12474            7375.79
                12476            6816.42
                12477           13219.74
                12590            9864.26
                12621           13689.67
                12626            6620.48
                12705            6868.29
                12709            9736.52
Japan           12753           21429.39
                12798            6977.04
Netherlands     14646          280206.02
Norway          12433           13375.87
Portugal        12757            7857.10
                12766            6989.63
Singapore       12744           21279.29
Spain           12540           13400.24
                12557           11990.96
Sweden          17404           31906.82
Switzerland     12357            6207.67
                12409           11072.67
                12451            9035.52
United Kingdom  12346           77183.60
                12748           33719.73
                12830            6814.64
                12901           17654.54
                12921           16587.09
                12931           42055.96
                12939           11581.80
                12971           11189.91
                12980            7374.90
                12989            6924.11
                13001            9818.82
                13018            7277.70
                13027            6912.00
                13078           12627.94
                13081           28337.38
                13089           58825.83
                13090            8910.82
                13093            7832.47
                13097            6010.42
                13098           28882.44
                13102            6424.09
                13113           12245.96
                13199            9817.12
                13209            5891.61
                13225            6463.84
                13263            7454.07
                13316            5977.73
                13319           10845.55
                13324            7121.95
                13340           10736.48
                13408           28117.04
                13418            6127.43
                13458            9646.57
                13488            8968.31
                13534            5870.08
                13576            6717.34
                13629            7792.36
                13668            6363.59
                13694           65039.62
                13709            6328.05
                13767           17220.36
                13777           25977.16
                13798           37153.85
                13854            8109.52
                13871            6403.25
                13881           17159.52
                13969            9053.94
                13985            7060.13
                14031           17286.86
                14051           15610.39
                14056            8214.65
                14057            6217.96
                14060            6140.42
                14062           10464.85
                14088           50491.81
                14096           65164.79
                14101            5976.79
          

In [40]:
countryVIP.groupby(['Country']).count().sort_values(by='amount_spent', ascending=False).iloc[0:1]

,amount_spent
Country,
United Kingdom,177


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [41]:

countryTotal= countrygroup[countrygroup['amount_spent'] > 1661.640]


countryTotal

amount_spent
Country        CustomerID              
Australia      12388            2780.66
               12415          124914.53
               12424            1760.96
               12431            5514.67
Austria        12360            2662.06
...                                 ...
United Kingdom 18259            2338.60
               18260            2643.20
               18272            3078.58
               18283            2094.88
               18287            1837.28

[1085 rows x 1 columns]

In [42]:
countryTotal.groupby(['Country']).count().sort_values(by='amount_spent', ascending=False).iloc[0:1]

,amount_spent
Country,
United Kingdom,932
